In [ ]:
!pip install -r requirements.txt

In [ ]:
#  # ДАННЫЙ КОД НУЖЕН ДЛЯ ПОДГОТОВКИ ОКРУЖЕНИЯ В ПЕРВЫЙ РАЗ
#  
#  import sys
#  import torch
#  from manage_dataset_structure import decompose_json
#  sys.path.append('src')
#  from wiki_bench import WikiBench
#  from sentence_transformers import SentenceTransformer
#  
#  decompose_json() # json -> файлы
#  
#  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#  encoder = SentenceTransformer('sergeyzh/BERTA').to(device)
#  
#  bench = WikiBench(
#      key='',
#      url='',
#      model_name='',
#      device=device,
#      encoder=encoder
#  )
#  
#  bench.prepare_env()

In [1]:
import sys
import logging
import torch
sys.path.append('src')
from wiki_bench import WikiBench
from sentence_transformers import SentenceTransformer

with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

logging.getLogger('sentence_transformers.SentenceTransformer').setLevel(logging.ERROR)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = SentenceTransformer('sergeyzh/BERTA').to(device)

bench = WikiBench(
    key=key,
    url=url,
    model_name='ruadapt-qwen3-4b', # example model name
    device=device, # 'cuda'
    encoder=encoder, # 'sergeyzh/BERTA' - default
    number_of_articles=3 # число статей для подсчета
)

# подготовка окружения (корпус уже должен быть готов!)
bench.load_enviroment()

# Ранжирование источников
score_query = await bench.rank_query()

# Генерация плана
score_outline = await bench.rank_outline(
    neighbor_count=0,
    description_mode=0,
    clusterization_with_hint=True             
)

# Генерация секций
score_sections = await bench.rank_sections()

resource module not available on Windows


18:31:23 | INFO | wikibench | WikiBench initialized: model=ruadapt-qwen3-4b, articles=3
18:31:23 | INFO | wikibench | Loading enviroment...
18:31:23 | INFO | wikibench | Enviroment loaded!
18:31:23 | INFO | wikibench | Stage: rank_query started
rank_query: 100%|███████████████████████████████████████████████████████████████████| 3/3 [01:17<00:00, 25.95s/article]
18:32:41 | INFO | wikibench | Final result for stage rank_query: (0.9810372896996004, 0.8924301290215108)
18:32:41 | INFO | wikibench | Stage: rank_outline started: neighbor_count=0, description_mode=0, clusterization_with_hint=True


                  NDCG  R-Precision
Atomic Heart  0.998011     0.942029
ChatGPT       0.953926     0.838710
F-35          0.991175     0.896552


rank_outline: 100%|█████████████████████████████████████████████████████████████████| 3/3 [00:26<00:00,  8.78s/article]
18:33:07 | INFO | wikibench | Final result for stage rank_outline: P=0.5226 [0.4759; 0.5563] | R=0.6499 [0.6100; 0.7220] | F=0.5782 [0.5376; 0.6150]
18:33:07 | INFO | wikibench | Stage: rank_sections started


              Precision    Recall  F1-Score
Atomic Heart   0.475889  0.617766  0.537625
ChatGPT        0.535635  0.722001  0.615009
F-35           0.556281  0.609997  0.581902


rank_sections: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:58<00:00, 19.39s/article]
18:34:06 | INFO | wikibench | Final result for stage rank_sections: P=0.5244 [0.4694; 0.5739] | R=0.6156 [0.5394; 0.6982] | F=0.5532 [0.5030; 0.6016] | Rouge=0.5532 [0.5030; 0.6016] | BLEU=0.5532 [0.5030; 0.6016]


 article_id        topic  precision   recall       f1  rouge_l     bleu  num_sections
          1 Atomic Heart   0.459484 0.544824 0.490926 0.150626 0.044010             5
          2      ChatGPT   0.597180 0.734869 0.648931 0.294507 0.120031             4
          3         F-35   0.527178 0.601580 0.543679 0.185245 0.036245             7


In [1]:
import sys
import logging
import torch
sys.path.append('src')
from wiki_bench import WikiBench
from sentence_transformers import SentenceTransformer

with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

logging.getLogger('sentence_transformers.SentenceTransformer').setLevel(logging.ERROR)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = SentenceTransformer('sergeyzh/BERTA').to(device)

resource module not available on Windows


In [2]:
from openai_utils import AsyncList

async def run_bench_for_model(
    url: str,
    key: str,
    model_name: str,
    device = None,
    encoder = None,
    number_of_articles: int = 1,
    neighbor_count: int = 0,
    description_mode: int = 1,
    clusterization_with_hint: bool = True,
    main_path: str = 'results',
    errors_path: str = 'errors' 
    ):

    bench = WikiBench(
        url=url, 
        key=key, 
        model_name=model_name, 
        device=device, 
        encoder=encoder, 
        number_of_articles=number_of_articles,
        main_path=main_path,
        errors_path=errors_path
    )

    bench.load_enviroment()
    
    # ---------- запросы ----------
    await bench.rank_query()
    
    # ---------- планы статей ----------
    await bench.rank_outline(
        neighbor_count=0,
        description_mode=0,
        clusterization_with_hint=True
    )
    
    # ---------- секции статей ----------
    await bench.rank_sections()

    print(f'[{model_name}] — готово. Результаты: {main_path}')

async def run_all_models(
    url: str,
    key: str,
    device,
    encoder,
    number_of_articles: int = 1,
    neighbor_count: int = 0,
    description_mode: int = 1,
    clusterization_with_hint: bool = True,
    main_path: str = 'results',
    errors_path: str = 'errors'
):
    models = AsyncList()
    model_names = [
        'ruadapt-qwen3-4b',
        'RefalMachine/RuadaptQwen3-32B-Instruct-v2',
        'Qwen3-235B-A22B-Instruct-2507',
        'DeepSeek V3',
        'tpro-2'
    ]
    for model_name in model_names:
        models.append(run_bench_for_model(
            url=url,
            key=key,
            model_name=model_name,
            device=device,
            encoder=encoder,
            main_path=main_path,
            errors_path=errors_path,
            neighbor_count=neighbor_count,
            description_mode=description_mode,
            clusterization_with_hint=clusterization_with_hint,
            number_of_articles=number_of_articles
        ))

    await models.complete_couroutines(batch_size=len(model_names))
    models = await models.to_list()

In [3]:
await run_all_models(
    url=url,
    key=key,
    device=device,
    encoder=encoder,
    main_path='main_results',
    errors_path='errors',
    neighbor_count=0,
    description_mode=0,
    clusterization_with_hint=True,
    number_of_articles=100
)

00:07:04 | INFO | wikibench | WikiBench initialized: model=ruadapt-qwen3-4b, articles=100
00:07:04 | INFO | wikibench | Loading enviroment...
00:07:04 | INFO | wikibench | Enviroment loaded!
00:07:04 | INFO | wikibench | Stage: rank_query started
rank_query:   0%|                                                                         | 0/100 [00:00<?, ?article/s]00:07:04 | INFO | wikibench | WikiBench initialized: model=RefalMachine/RuadaptQwen3-32B-Instruct-v2, articles=100
00:07:04 | INFO | wikibench | Loading enviroment...
00:07:05 | INFO | wikibench | Enviroment loaded!
00:07:05 | INFO | wikibench | Stage: rank_query started

rank_query:   0%|                                                                         | 0/100 [00:00<?, ?article/s]00:07:05 | INFO | wikibench | WikiBench initialized: model=Qwen3-235B-A22B-Instruct-2507, articles=100
00:07:05 | INFO | wikibench | Loading enviroment...
00:07:06 | INFO | wikibench | Enviroment loaded!
00:07:06 | INFO | wikibench | Stage: r

                             NDCG  R-Precision
Atomic Heart             0.998140     0.943662
ChatGPT                  0.934747     0.774194
F-35                     0.992273     0.902098
Minecraft                0.991747     0.889610
Биткойн                  0.994786     0.925170
...                           ...          ...
Толстой, Лев Николаевич  0.992462     0.910138
Трамп, Дональд           0.946161     0.733333
Человек разумный         0.585450     0.185185
Шизофрения               0.990597     0.879747
Экономика России         0.810599     0.438202

[97 rows x 2 columns]


rank_outline:   0%|                                                                       | 0/100 [00:00<?, ?article/s]



rank_outline:  20%|████████████▍                                                 | 20/100 [04:33<18:07, 13.59s/article]



rank_outline:  24%|██████████████▉                                               | 24/100 [05:25<17:19, 13.68s/article]


rank_outline:  27%|████████████████▋                                             | 27/100 [06:15<18:31, 15.22s/article]

rank_outline:  33%|████████████████████▍                                         | 33/100 [07:24<13:18, 11.92s/article]

rank_outline:  37%|██████████████████████▉                                       | 37/100 [08:05<11:04, 10.55s/article]


rank_outline:  38%|███████████████████████▌                                      | 38/100 [08:21<12:29, 12.10s/article]

rank_outline:  40%|████████████████████████▊                                     | 40/100 [08:52<13:59, 14.00s/article]

rank_outline:  44%|███████

                         Precision    Recall  F1-Score
Atomic Heart              0.449635  0.592766  0.511374
ChatGPT                   0.445978  0.702078  0.545464
F-35                      0.535376  0.614134  0.572057
Minecraft                 0.567858  0.614127  0.590087
Биткойн                   0.575329  0.629191  0.601056
...                            ...       ...       ...
Толстой, Лев Николаевич   0.534950  0.559947  0.547163
Трамп, Дональд            0.557033  0.562939  0.559970
Человек разумный          0.576725  0.536693  0.555989
Шизофрения                0.588243  0.672234  0.627440
Экономика России          0.620023  0.615391  0.617698

[100 rows x 3 columns]


rank_sections:   2%|█▏                                                            | 2/100 [00:28<22:41, 13.90s/article]


rank_sections:   4%|██▍                                                           | 4/100 [01:44<48:50, 30.53s/article]


rank_sections:   9%|█████▍                                                      | 9/100 [06:09<1:05:10, 42.98s/article]


rank_sections:  11%|██████▍                                                    | 11/100 [08:03<1:17:21, 52.15s/article]


rank_sections:  17%|██████████▎                                                  | 17/100 [10:52<44:31, 32.18s/article]

rank_sections:  18%|██████████▉                                                  | 18/100 [11:28<45:26, 33.25s/article]



rank_sections:  19%|███████████▏                                               | 19/100 [14:01<1:33:27, 69.22s/article]



rank_query:  35%|████████████████████▋                                      | 35/100 [2:40:09<5:46:24, 319.76s/article]

rank_query:  30%|███████

...


rank_outline:  83%|█████████████████████████████████████████████████▊          | 83/100 [1:05:44<11:47, 41.64s/article]



rank_outline:  88%|████████████████████████████████████████████████████▊       | 88/100 [1:09:39<09:05, 45.44s/article]



rank_outline:  94%|████████████████████████████████████████████████████████▍   | 94/100 [1:14:02<03:49, 38.26s/article]



rank_outline:  95%|█████████████████████████████████████████████████████████   | 95/100 [1:14:56<03:35, 43.01s/article]



rank_outline:  98%|██████████████████████████████████████████████████████████▊ | 98/100 [1:18:01<01:44, 52.18s/article]



rank_outline: 100%|███████████████████████████████████████████████████████████| 100/100 [1:20:08<00:00, 48.09s/article]
05:18:33 | INFO | wikibench | Final result for stage rank_outline: P=0.6095 [0.6017; 0.6179] | R=0.6532 [0.6422; 0.6643] | F=0.6296 [0.6215; 0.6380]
05:18:33 | INFO | wikibench | Stage: rank_sections started


                         Precision    Recall  F1-Score
Atomic Heart              0.600968  0.710526  0.651171
ChatGPT                   0.568386  0.740158  0.642997
F-35                      0.593609  0.686823  0.636823
Minecraft                 0.611912  0.667069  0.638301
Биткойн                   0.642933  0.673441  0.657833
...                            ...       ...       ...
Толстой, Лев Николаевич   0.597928  0.659990  0.627428
Трамп, Дональд            0.592049  0.553547  0.572151
Человек разумный          0.597557  0.577498  0.587356
Шизофрения                0.585038  0.655489  0.618263
Экономика России          0.658028  0.621845  0.639425

[100 rows x 3 columns]


rank_sections:   4%|██▎                                                        | 4/100 [09:38<4:21:07, 163.21s/article]



rank_sections:  15%|████████▋                                                 | 15/100 [35:03<2:58:01, 125.67s/article]



rank_sections:  16%|█████████▍                                                 | 16/100 [35:26<2:12:17, 94.50s/article]



rank_sections:  17%|██████████                                                 | 17/100 [37:00<2:10:30, 94.34s/article]



rank_sections:  19%|███████████▏                                               | 19/100 [40:08<2:05:51, 93.23s/article]



rank_query:  84%|███████████████████████████████████████████████████▏         | 84/100 [5:31:52<54:52, 205.76s/article]



rank_sections:   6%|███▌                                                       | 6/100 [23:08<7:42:55, 295.49s/article]



rank_sections:  23%|█████████████▌                                             | 23/100 [47:41<1:56:04, 90.44s/article]



rank_sections:  

                             NDCG  R-Precision
Atomic Heart             1.000000     1.000000
ChatGPT                  0.982467     0.875000
F-35                     0.997004     0.951724
Minecraft                0.996242     0.934211
Биткойн                  0.996409     0.918750
...                           ...          ...
Толстой, Лев Николаевич  0.996487     0.942263
Трамп, Дональд           0.982277     0.822951
Человек разумный         0.854357     0.450704
Шизофрения               0.998171     0.963415
Экономика России         0.943502     0.638889

[97 rows x 2 columns]




rank_outline:   0%|                                                                       | 0/100 [00:00<?, ?article/s]